In [7]:
#probably need to update these imports to the new name, PyHAT
from point_spectra_gui.util.spectral_data import spectral_data as spectral_data
import libpysat.regression.cv as cv2
import glob
import os
import warnings
import numpy as np

# from sklearn.linear_model import RANSACRegressor as RANSAC
import pandas as pd
from libpysat.regression.regression import regression
from sklearn.linear_model import enet_path, lasso_path
from sklearn.linear_model.base import _pre_fit
from sklearn.utils.validation import check_X_y, check_array
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import ParameterGrid
warnings.filterwarnings('ignore')
import time
import copy
import itertools
# Test for commit

In [8]:
# This function takes the data as input, and returns a spectral data object
def CCAM_CSV(input_data, ave=True):
    
    #These try/excepts are clunky but get the job done
    try:
        df = pd.read_csv(input_data, header=14, engine='c')
        cols = list(df.columns.values)
        df.columns = [i.strip().replace('# ', '') for i in cols]  # strip whitespace from column names
        df.set_index(['wave'], inplace=True)  # use wavelengths as indices
        # read the file header and put information into the dataframe as new columns
        metadata = pd.read_csv(input_data, sep='=', nrows=14, comment=',', engine='c', index_col=0, header=None)
    except:
        try:  # handle files with an extra header row containing temperature
            df = pd.read_csv(input_data, header=15, engine='c')
            cols = list(df.columns.values)
            df.columns = [i.strip().replace('# ', '') for i in cols]  # strip whitespace from column names
            df.set_index(['wave'], inplace=True)  # use wavelengths as indices
            # read the file header and put information into the dataframe as new columns
            metadata = pd.read_csv(input_data, sep='=', nrows=15, comment=',', engine='c', index_col=0, header=None)
        except:  # handle files with an extra header row containing temperature and target name
            df = pd.read_csv(input_data, header=16, engine='c')
            cols = list(df.columns.values)
            df.columns = [i.strip().replace('# ', '') for i in cols]  # strip whitespace from column names
            df.set_index(['wave'], inplace=True)  # use wavelengths as indices
            # read the file header and put information into the dataframe as new columns
            metadata = pd.read_csv(input_data, sep='=', nrows=16, comment=',', engine='c', index_col=0, header=None)

    if ave:
        df = pd.DataFrame(df['mean'])
    else:
        df = df.drop(['mean', 'median'], axis=1)
    df.index = [['wvl'] * len(df.index),
                df.index.values.round(4)]  # create multiindex so spectra can be easily extracted with a single key
    df = df.T  # transpose so that each spectrum is a row

    # remove extraneous stuff from the metadataindices
    metadata.index = [i.strip().strip('# ').replace(' FLOAT', '').lower() for i in metadata.index.values]
    metadata = metadata.T

    # extract info from the file name
    fname = os.path.basename(input_data)
    metadata['sclock'] = fname[4:13]
    metadata['seqid'] = fname[25:34].upper()
    metadata['Pversion'] = fname[34:36]

    # duplicate the metadata for each row in the df
    if not ave:
        metadata = metadata.append([metadata] * (len(df.index) - 1), ignore_index=True)
    metadata.index = df.index  # make the indices match
    metadata.columns = [['meta'] * len(metadata.columns), metadata.columns.values]  # make the columns into multiindex
    df = pd.concat([metadata, df], axis=1)  # combine the spectra with the metadata
    return df

# alkali_cv.py script
def run_cross():
    #probably need to update these imports to the new name, PyHAT
    best_settings = []
    models = []

    #specify the list of elements that we want to develop models for
    elements = ['Na2O','K2O']

    #set output path
    outpath = "/home/egault/Desktop/fork1/PySAT-master/ConstrainedData/"

    #set source data path
    sourcepath = outpath

    # set the composition ranges to consider. This allows cross validation to test out all of the different submodels
    # that will be used in the eventual calibration
    # yranges = [[0,100], [0,1.5], [1,6], [5,100]]
#     yranges = [[240.1, 342.2], [380.1, 469.3], [472.0, 906.5]]
    
    yranges = [[0,100]]

    #set the list of methods to try
    methods = [
                'PLS']
    # methods = [
    #             'PLS',
    #             'Elastic Net',
    #             'LASSO',
    #             'Ridge',
    #             'GP',
    #             'OLS',
    #             'BRR',
    #             'LARS',
    #             'OMP'
    #             ]
    #create a log-spaced set of alpha values to consider - this is used by a number of different methods
    alphas = np.logspace(np.log10(0.000000001), np.log10(0.001),
                                             num=20)

    # set the parameters to consider for each method. Each parameter is in a list, so more than one value can be specified.
    # The cross validation will evaluate every possible permutation of parameters, so beware of adding too many,
    # especially for slower methods
    params = {'PLS':{'n_components': [1,2,3],
                                  'scale': [False]}}
    # params = {'PLS':{'n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    #                               'scale': [False]},
    #             'Elastic Net':{ 'alpha': alphas,
    #                                 'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1.0],
    #                                 'fit_intercept': [True, False],
    #                                 'normalize': [False],
    #                                 'precompute': [True],
    #                                 'max_iter': [1000],
    #                                 'copy_X': [True],
    #                                 'tol': [0.0001],
    #                                 'warm_start': [True],
    #                                 'positive': [True, False],
    #                                 'selection': ['random']},
    #            'GP':{   'reduce_dim': ['PCA'],
    #                     'n_components': [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],
    #                     'regr': ['linear'],
    #                     'corr': ['squared_exponential'],
    #                     'storage_mode': ['light'],
    #                     'verbose': [True],
    #                     'theta0': [0.1],
    #                     'normalize': [True],
    #                     'optimizer': ['fmin_cobyla'],
    #                     'random_start': [5]},
    #            'OLS':{  'fit_intercept': [True, False]},
    #            'LASSO':{'alpha': list(alphas),
    #                     'fit_intercept': [True, False],
    #                     'max_iter': [1000],
    #                     'tol': [0.0001],
    #                     'positive': [True, False],
    #                     'selection': ['random']},
    #            'BRR':{  'n_iter': [300],
    #                     'tol': [0.001],
    #                     'alpha_1': [0.000001],
    #                     'alpha_2': [0.000001],
    #                     'lambda_1': [0.000001],
    #                     'lambda_2': [0.000001],
    #                     'compute_score': [False],
    #                     'fit_intercept': [True, False],
    #                     'normalize': [False],
    #                     'copy_X': [True],
    #                     'verbose': [True]},
    #
    #            'LARS':{ 'fit_intercept': [True, False],
    #                     'verbose': [True],
    #                     'normalize': [False],
    #                     'precompute': ['auto'],
    #                     'n_nonzero_coefs': [1,5,10,50,100,200,400,800],
    #                     'copy_X': [True],
    #                     'fit_path': [False],
    #                     'positive': [True, False]},
    #            'OMP':{'n_nonzero_coefs': [1,5,10,50,100,200,400,800],
    #                   'fit_intercept': [True, False],
    #                   'normalize': [False],
    #                   'precompute': ['auto'],
    #                   },
    #            'Ridge':{'alpha': list(alphas),
    #                     'copy_X': [True],
    #                     'fit_intercept': [True, False],
    #                     'max_iter': [None],
    #                     'normalize': [False],
    #                     'solver': ['auto'],
    #                     'tol': [0.001],
    #                     'random_state': [None]}
    #             }
    
    # Filepath to the maskfile you would like to use
    maskfile = "/home/egault/Desktop/fork1/PySAT-master/mask_minors_noise_20160202_adjusted.csv"
    
    # Empty List to hold Cross Validation prediction keys produced by do_cv().
    cvpredictkeys = []
    
    # Use the read_excel() method if dealing with .xlsx reference target files.
    # This may require an additional import.

    # Obtain reference target file, then pack it into a dataframe.
    mars_reftar_file = "/home/egault/Desktop/fork1/PySAT-master/mars_reference_targets_simplified.csv"
    mars_reftar_df = pd.read_csv(mars_reftar_file)
    
    
    #Step through each of the elements listed
    for element in elements:
        #get the data normalized to 3 (each spectrometer separately) or to 1 (sum of all spectrometers)
        filenames = [sourcepath+"data1_"+element+"_train.csv",sourcepath+"data3_"+element+"_train.csv"]
        print("Filenames: ", filenames)
        #Step through each data file
        for file in filenames:
            # Read the data in, make it a "spectral_data" object
            data = spectral_data(pd.read_csv(file, header=[0, 1], verbose=True))
            #Step through each regression method
            for method in methods:
                paramstemp = list(ParameterGrid(params[method])) #get the parameters for this method
                cv_obj = cv2.cv(paramstemp) #set up cross validation across all permutations of parameters

                #do the cross validation for each composition range
                for yrange in yranges:
                    #set up an output file name that specifies the current composition range being considered
                    if file == filenames[0]:
                        outfile_root = 'data1_' + element + '_' + str(yrange[0])+'-'+str(yrange[1])+'_'
                    if file == filenames[1]:
                        outfile_root = 'data3_' + element + '_' + str(yrange[0])+'-'+str(yrange[1])+'_'

                    #apply yrange to filter the compositions used in the regression
                    y = np.array(data.df[('comp',element)])
                    match = np.squeeze((y > yrange[0]) & (y < yrange[1]))
                    datatemp = spectral_data(data.df.ix[match])
                    datatemp.mask(maskfile, 'wvl')
                    
                    # Try to create a cv_iterator object, as it is required to run the do_cv method.
                    try:
                        cv_iterator = LeaveOneGroupOut().split(datatemp.df['wvl'], datatemp.df[('comp', element)], datatemp.df[
                            ('meta', 'Folds')])  # create an iterator for cross validation based on the predefined folds
                        n_folds = LeaveOneGroupOut().get_n_splits(groups=datatemp.df[('meta', 'Folds')])

                    except:
                        print('***No folds found! Did you remember to define folds before running cross validation?***')
                    
                    #do the actual cross validation for the current method, element, and yrange
                    datatemp.df, cv_results, cvmodels, cvmodelkeys, cvpredictkeys = cv_obj.do_cv(datatemp.df, cv_iterator,
                                                                                  xcols = 'wvl',
                                                                                  ycol = ('comp', 'Na2O'), method=method,
                                                                                  yrange=yrange, calc_path=False,
                                                                                  n_folds = n_folds)

                    
                    # Filepath to read in training data
                    training_data_filepath = "/home/egault/Desktop/fork1/PySAT-master/FullData/"
                    
                    # Filepath to read in mars data
                    mars_filepath = "/home/egault/Desktop/fork1/PySAT-master/reference_data"
                    
                    # This grabs all .csv files and puts them in a list
                    allFiles = glob.glob(mars_filepath + "/*.csv")
                    # Iterates the entire list
                    for file_ in allFiles:
                            filename = os.path.splitext(file_)
                            filename = os.path.basename(filename[0])
                            
                            # Converts the .csv to readable format for the spectral data object
                            mars_df = CCAM_CSV(file_)
                            
                            # Converts the .csv to a spectral data object
                            mars_df_obj = spectral_data(mars_df)
                            
                            # Iterates over the modesls that were returned 
                            for model in cvmodels:
                                
                                # Normalization causes following error with certain mars files:
                                # ValueError: cannot handle a non-unique multi-index!
                                
                                # A stable file that is within the current reference target spreadsheet
                                # is named:CL5_433631305CCS_F0170000CCAM01407P1.csv
                                # This file was ideal for testing purposes.
                                
                                mars_df_obj.norm([[0, 1000]], 'wvl')
                                
                                
                                # Masks values based on the maskfile you passed
                                mars_df_obj.mask(maskfile, 'wvl')
                                
                                # Runs the model and predicts a value for the wavelengths 
                                predict = model.predict(mars_df_obj.df['wvl'])
                                
                                # Obtain minimum value for prediction check.
                                
                                # Get label of min value column.
                                min_col_key = element + ' (min)'
                                
                                # Prints the values
                                print("Predicted: ", predict)
    
                    # save cross validation results
                    filename = outpath + outfile_root + method + '_CV.csv'
                    cv_results['cv'].to_csv(filename)

                    # save cross validation predictions
                    filename = outpath + outfile_root + method + '_CV_results.csv'
                    temp = datatemp.df.drop('wvl', axis=1, level=0)                   
                    
                    
                    temp.to_csv(filename)

                pass

In [9]:
run_cross()


Filenames:  ['/home/egault/Desktop/fork1/PySAT-master/ConstrainedData/data1_Na2O_train.csv', '/home/egault/Desktop/fork1/PySAT-master/ConstrainedData/data3_Na2O_train.csv']
Tokenization took: 67.30 ms
Type conversion took: 137.32 ms
Parser memory cleanup took: 0.20 ms
Tokenization took: 62.50 ms
Type conversion took: 134.28 ms
Parser memory cleanup took: 0.14 ms
Tokenization took: 63.74 ms
Type conversion took: 133.23 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 60.64 ms
Type conversion took: 139.22 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 61.45 ms
Type conversion took: 138.01 ms
Parser memory cleanup took: 0.13 ms
Tokenization took: 64.32 ms
Type conversion took: 139.44 ms
Parser memory cleanup took: 0.13 ms
Tokenization took: 62.34 ms
Type conversion took: 143.51 ms
Parser memory cleanup took: 0.12 ms
Tokenization took: 61.75 ms
Type conversion took: 140.14 ms
Parser memory cleanup took: 0.12 ms
Tokenization took: 61.94 ms
Type conversion took: 144.02 ms

Type conversion took: 139.75 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 60.44 ms
Type conversion took: 132.60 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 62.94 ms
Type conversion took: 134.26 ms
Parser memory cleanup took: 0.14 ms
Tokenization took: 61.55 ms
Type conversion took: 139.11 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 62.40 ms
Type conversion took: 134.45 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 63.97 ms
Type conversion took: 134.94 ms
Parser memory cleanup took: 0.11 ms
Tokenization took: 8.30 ms
Type conversion took: 74.58 ms
Parser memory cleanup took: 0.35 ms
{'n_components': 1, 'scale': False}
[{'n_components': 1, 'scale': False}]
PLSRegression(copy=True, max_iter=500, n_components=1, scale=False, tol=1e-06)
440
[{'n_components': 1, 'scale': False}]
PLSRegression(copy=True, max_iter=500, n_components=1, scale=False, tol=1e-06)
434
[{'n_components': 1, 'scale': False}]
PLSRegression(copy=True, max_iter=500, n_c

In [10]:
train_df

NameError: name 'train_df' is not defined